## Web Scraping Terras Indigenas

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
def extrair_uf_da_pagina_detalhe(url_detalhe):
    """
    Acessa a página de detalhes de uma terra indígena, encontra todos os
    elementos com a classe 'info-box-number', coleta todas as siglas de UF
    válidas e as retorna como uma string única, separada por vírgulas.
    """
    siglas_uf_brasil = [
        'AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS',
        'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC',
        'SP', 'SE', 'TO'
    ]
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    ufs_encontradas = [] # Lista para armazenar todas as UFs da página
    
    try:
        response = requests.get(url_detalhe, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        info_boxes = soup.find_all('span', class_='info-box-number')
        
        for box in info_boxes:
            texto_box = box.text.strip().upper()
            
            # Se o texto for uma UF válida e ainda não estiver na lista, adicione-o
            if texto_box in siglas_uf_brasil and texto_box not in ufs_encontradas:
                ufs_encontradas.append(texto_box)

    except requests.exceptions.RequestException as e:
        print(f"  -> Erro ao acessar {url_detalhe}: {e}")
        return "Erro na busca" # Retorna erro se a página não puder ser acessada
    
    # Se a lista de UFs não estiver vazia, junte os itens com ", "
    if ufs_encontradas:
        return ", ".join(ufs_encontradas)
    
    return "UF não localizada"

In [3]:
# URL da página principal com a tabela
url_base = 'https://terrasindigenas.org.br'
url_tabela = f'{url_base}/pt-br/brasil'

print("Iniciando o web scraping da página principal...")

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

Iniciando o web scraping da página principal...


In [4]:
try:
    response_tabela = requests.get(url_tabela, headers=headers)
    response_tabela.raise_for_status()
    soup_tabela = BeautifulSoup(response_tabela.text, 'html.parser')

    tabela = soup_tabela.find('table', class_='table table-striped tablesorter')
    
    links_por_nome = {}
    if tabela:
        for linha in tabela.find_all('tr')[1:]:
            coluna_nome = linha.find('td')
            if coluna_nome:
                nome_ti = coluna_nome.text.strip()
                link_tag = coluna_nome.find('a')
                if link_tag and link_tag.has_attr('href'):
                    links_por_nome[nome_ti] = f"{url_base}{link_tag['href']}"
    
    print(f"Encontrados {len(links_por_nome)} links para as terras indígenas.")

    # Carregar o arquivo CSV
    df = pd.read_csv('terras_indigenas.csv')

    # Criar a coluna 'UF' se ela não existir
    if 'UF' not in df.columns:
        df['UF'] = '' # Inicializa com string vazia ou outro valor padrão

    # Garante que a coluna UF seja do tipo 'object' (texto) para evitar erros
    df['UF'] = df['UF'].astype(object)

    # Preencher a coluna UF
    for index, row in df.iterrows():
        nome_ti = row['Nome']
        if nome_ti in links_por_nome:
            print(f"Processando: {nome_ti}...")
            url_detalhe = links_por_nome[nome_ti]
            uf = extrair_uf_da_pagina_detalhe(url_detalhe)
            df.at[index, 'UF'] = uf
            print(f"  -> UF encontrada: {uf}")
            time.sleep(1) 
        else:
            print(f"Aviso: Não foi encontrado um link para '{nome_ti}' na página.")
            df.at[index, 'UF'] = 'Link não encontrado'

    # Salvar o DataFrame atualizado
    df.to_csv('terras_indigenas_com_uf.csv', index=False, encoding='utf-8-sig')
    print("\nProcesso finalizado! O arquivo 'terras_indigenas_com_uf.csv' foi criado.")

except requests.exceptions.RequestException as e:
    print(f"Não foi possível acessar a página principal: {e}")
except FileNotFoundError:
    print("Arquivo 'terras_indigenas.csv' não encontrado. Verifique se ele está na mesma pasta que o script.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

Encontrados 776 links para as terras indígenas.
Processando: Acapuri de Cima...
  -> UF encontrada: AM
Processando: Acapuri do Meio...
  -> UF encontrada: AM
Processando: Acimã...
  -> UF encontrada: AM
Processando: Aconã...
  -> UF encontrada: AL
Processando: Água Grande...
  -> UF encontrada: RS
Processando: Água Preta/Inari...
  -> UF encontrada: AM
Processando: Águas Belas...
  -> UF encontrada: BA
Processando: Águas Claras...
  -> UF encontrada: SC
Processando: Aldeia Beija Flor...
  -> UF encontrada: AM
Processando: Aldeia Cerroy...
  -> UF encontrada: MS
Processando: Aldeia Katurama...
  -> UF encontrada: MG
Processando: Aldeia Kondá...
  -> UF encontrada: SC
Processando: Aldeia Limão Verde...
  -> UF encontrada: MS
Processando: Aldeia Renascer (Ywyty Guasu)...
  -> UF encontrada: SP
Processando: Aldeia Serra do Couro/Aldeia Serra do Cachimbo...
  -> UF encontrada: BA
Processando: Aldeias Santa Maria e Nova Bacaba (Baixo Rio Jatapu)...
  -> UF encontrada: AM
Processando: Aldeia 

In [5]:
links_por_nome

{'Acapuri de Cima': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/4184',
 'Acapuri do Meio': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/6296',
 'Acimã': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/3935',
 'Aconã': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/5106',
 'Água Grande': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/5122',
 'Água Preta/Inari': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/3914',
 'Águas Belas': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/3568',
 'Águas Claras': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/6203',
 'Aldeia Beija Flor': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/5326',
 'Aldeia Cerroy': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/6294',
 'Aldeia Katurama': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/6204',
 'Aldeia Kondá': 'https://terrasindigenas.org.br/pt-br/terras-indigenas/4115',
 'Aldeia Limão Verde': 'https://terrasindigenas.o